In [14]:
import os

topic_file = os.getenv("TOPIC_PATH")
users_file = os.getenv("USERS_PATH")
ratings_file = os.getenv("RATINGS_PATH")

topic_file = "/data/books_data/books.csv"
users_file = "/data/books_data/users.csv"
ratings_file = "/data/books_data/rating.csv"

In [16]:
import pandas as pd
topics = pd.read_csv(topic_file)

In [18]:
topics["ID"] = pd.to_numeric(topics["ID"], errors='coerce')

In [19]:
topics.head()

,ID,NAME,CATEGORY
0,6.097313e+07,Carlo D'Este,Decision in Normandy
1,4.251764e+08,Robert Cowley,What If?: The World's Foremost Military Histor...
2,4.522645e+08,Toni Morrison,Beloved (Plume Contemporary Fiction)
3,6.098046e+08,The Onion,Our Dumb Century: The Onion Presents 100 Years...
4,1.879384e+09,J. R. Parrish,If I'd Known Then What I Know Now: Why Not Lea...


In [20]:
ratings = pd.read_csv(ratings_file)
ratings.head()

,UserID,MovieID,Rating
0,1,6264,7.0
1,1,4350,7.0
2,1,6252,5.0
3,1,202,9.0
4,1,6266,6.0


In [21]:
users = pd.read_csv(users_file)
users.head()

,Gender,Age,UserID
0,"minneapolis, minnesota, usa",24,1
1,"san diego, california, usa",20,2
2,"novinger, missouri, usa",16,3
3,"sonoma, california, usa",34,4
4,"berkeley, california, usa",23,5


In [22]:
user_rating = pd.merge(users,ratings,on='UserID')
user_rating_mv = pd.merge(user_rating, topics, left_on="MovieID", right_on="ID") 

In [23]:
user_rating_mv

,Gender,Age,UserID,MovieID,Rating,ID,NAME,CATEGORY


In [8]:
#df = pd.DataFrame().reset_index()

top_movies = user_rating_mv[['UserID','Gender','Age', 'NAME']].groupby(['Gender','Age','NAME']).count().reset_index()

In [9]:
df1 = top_movies.query('Gender=="M" & Age=="25"').sort_values('UserID', ascending=False)
df1['NAME'].head(10)

24127                               American Beauty (1999)
26955    Star Wars: Episode V - The Empire Strikes Back...
27081                    Terminator 2: Judgment Day (1991)
26954            Star Wars: Episode IV - A New Hope (1977)
26956    Star Wars: Episode VI - Return of the Jedi (1983)
25983                                   Matrix, The (1999)
26721                           Saving Private Ryan (1998)
26810                     Silence of the Lambs, The (1991)
25687                                 Jurassic Park (1993)
26011                                  Men in Black (1997)
Name: NAME, dtype: object

In [10]:
def fetch_top_movies(age, gender):
    global top_movies
    qry = 'Gender=="'+gender+'" & Age=="'+age+'"'
    df1 = top_movies.query(qry).sort_values('UserID', ascending=False)
    return df1.head(10).NAME.to_list()

In [11]:
print(fetch_top_movies('25','F'))

['American Beauty (1999)', 'Shakespeare in Love (1998)', 'Silence of the Lambs, The (1991)', 'Princess Bride, The (1987)', 'Fargo (1996)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Being John Malkovich (1999)', 'Sixth Sense, The (1999)', 'Back to the Future (1985)']


In [12]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours
import numpy
import pandas as pd

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    print(payload)     
    global top_movies
    qry = 'Gender=="'+payload["gender"]+'" & Age=="'+payload["age"]+'"'
    df1 = top_movies.query(qry).sort_values('UserID', ascending=False)
    return df1.head(10).NAME.to_list()
    return prediction.tolist()

/opt/conda/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [13]:
modl_ob = register_model(None, score, name="Trending_Topic", 
                    description="Trending topics",
                    flavour=MLModelFlavours.sklearn)

In [17]:
print(modl_ob)

None
